In [13]:
import glob

# Importing drive method from colab for accessing google drive
from google.colab import drive

In [14]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import glob
import numpy as np
import pandas as pd 

In [16]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [17]:
import copy 

In [68]:
class Model() :
    network = [[1 for i in range(4)] for j in range(100000)]
    it =  0 

    def input(self,input) :
        self.input = input 
        
    def output(self,output) :
        self.output = output 
        
    def add(self,input_shape , prev_shape , activation_func) :
            weight = [[random() for i in range(prev_shape)] for j in range(input_shape)]
            neuron = [random() for i in range(input_shape)]
            bias = random()

            self.network[self.it][0],self.network[self.it][1],self.network[self.it][2],self.network[self.it][3] =  weight,neuron,bias,activation_func
            self.it+=1 
            
    
            
    
    

In [69]:
def conv2d(image,kernel) :
    
    #kernel.size = (3,3) ; 
    length = len(image)
    #print(length)
    output = [[[0,0,0]  for j in range(length-3)] for i in range(length-3) ]
    for i in range(0,length-3,1) :
        for j in range(0,length-3,1) :
            for k in range(3):
                ans = 0 
                for a in range(3):
                    for b in range(3) :
                        ans += image[i+a][j+b][k]*kernel[a][b]
                #print(ans)
                output[i][j][k] =  ans 
                
                
    return output

# (image[3:6][0:3][0]*kernel).sum()
     

In [70]:
filter = [[[1/9,1/9,1/9],[1/9,1/9,1/9],[1/9,1/9,1/9]],[[0,-1,0],
                                                       [-1,5,-1],[0,-1,0]],[[1 ,0,1],[-2,0,2],[-1,0,1]],[[1,-2,-1],[0,0,0],[1,2,1]]]

In [71]:
def pooling(feature_map, size=2, stride=2):    
    pool_out = np.zeros((np.uint16((feature_map.shape[0]-size+1)/stride),  
                              np.uint16((feature_map.shape[1]-size+1)/stride),  
                              feature_map.shape[-1]))  
    for map_num in range(feature_map.shape[-1]):  
        r2 = 0  
        for r in np.arange(0,feature_map.shape[0]-size-1, stride):  
            c2 = 0  
            for c in np.arange(0, feature_map.shape[1]-size-1, stride):  
                pool_out[r2, c2, map_num] = np.max(feature_map[r:r+size,  c:c+size])  
                c2 = c2 + 1  
            r2 = r2 +1 
            
    return pool_out
                

FORWARD PROPAGATION

```
# This is formatted as code
```



In [72]:
def active_sum(input,weight,bias) :
    activation = bias 
    for i in range(len(weight)) : 
        activation += (weight[i]*input[i]) 
        
    return activation

In [73]:
def sigmoid(input) :
    return 1.0/(1.0 + np.exp(-input))

In [74]:
def ReLU(input) :
    if(input >0) :
        return input 
    return (0.01*input)

In [75]:
def Softmax(input) :
  output = [0 for i in range(len(input))]
  sum =  0 
  for i in range(len(input)) :
    sum += np.exp(input[i]) 
  for i in range(len(input)) :
    output[i] = (np.exp(input[i]))/sum 

  return output

In [76]:
def Activation(sum_active,function) :
    if(function == 'ReLU'):
        return ReLU(sum_active)
    if(function == 'sigmoid') :
        return sigmoid(sum_active)
    if(function == 'softmax') :
      return Softmax(sum_active)

In [115]:
def forward_prop(network,input,bias,it) :
    #print(it)
    my_input = input
    my_bias = bias 
    #print(my_input)
    for i in range(it-1) :  #Each hidden layer
        sum_active = [0 for i in range(len(network[i][1]))]
        for j in range(len(network[i][1])): #For each neuron 
            sum_active[j] = active_sum(network[i][0][j],my_input,my_bias)
        
        sum_active = normalise(sum_active)
        for j in range(len(network[i][1])) :
          activation = Activation(sum_active[j],network[i][3])      
          network[i][1][j] = activation
        
        my_input = network[i][1]
        my_bias = network[i][2]         
            
    #### For last layer 
    sum_active1 = [0 for i in range(len(network[it-1][1]))]
    for j in range(len(network[it-1][1])) :
        sum_active1[j] = active_sum(network[it-1][0][j],my_input,my_bias)
    
    sum_active = normalise(sum_active)
    output = Softmax(sum_active1)
    network[it-1][1] = output 
        
            
    #output = Softmax(sum_active)
    #for j in range(len(network[it-1][1])): #For each neuron 
     #   network[it-1][1][j] = output[j]
                     
    return network

In [78]:
def sig_derivative(input) :
    
    return sigmoid(input)*(1.0  - sigmoid(input))

In [79]:
def rel_derivative(input) :
    if(input >0) :
        return 1 
    return (-0.01)

In [80]:
def soft_derivative(x) : # Complete layer is passed 
    y = [0 for i in range(len(x))] 
    for i in range(len(x)) :
        for j in range(len(x)) :
            if(i != j) :
                y[i] += x[i]*x[j]*(-1) 
            else :
                y[i] += x[i]*(1-x[i]) 
                
    return y

In [81]:
def derivative(input,function) :

    if(function == 'ReLU') :
       ## print("ReLu")
        return rel_derivative(input)
    if(function == 'sigmoid') :
        #print("Sigmoid")
        return sig_derivative(input)

      
        

In [90]:
def back_prop(network,input,output,learning_rate,bias,it) :
    error = [[1 for i in range(1000)] for j in range(it)]
    #For last layer 
    it-=1 
    sum_det = 0 #For bias 

    ### Calculation error and derivative for Softmax(last layer)
    error[it] =  soft_derivative(network[it][1]) 
    
    for j in range(len(error[it])) :
        sum_det += error[it][j] 

    for i in range(len(network[it][1])) : #For each neuron  
        #Now adjusting all weights to this neuron 
        delta = error[it][i]
        for j in range(len(network[it][0][i])) :
            network[it][0][i][j] += (learning_rate*delta*network[it-1][1][j])
            
            
    network[it-1][2] *= (1 + learning_rate*sum_det)
    
    it-=1 
    
    
    ## Now doing for all layers but first 
    while(it != 0) :
        sum_det = 0
        for i in range(len(network[it][1])) : #For each neuron 
            
            err = 0 #Calculating err for each neuron which is summation(error[it+1][j]*w[j][i])
            
            for j in range(len(network[it+1][1])) :
                err += (error[it+1][j]*network[it+1][0][j][i])
                
            
            error[it][i] = err 
            delta = error[it][i]*derivative(network[it][1][i],network[it][3]) 
            error[it][i] = delta
            sum_det += delta
            for j in range(len(network[it][0][i])) :
                network[it][0][i][j] += (learning_rate*delta*network[it-1][1][j])
            
            
        network[it-1][2] *= (1 + learning_rate*sum_det)

        it-=1 
        
        
    #Now for last layer . Here the only change will be that instead of network we'll take input and input bias 
    
    sum_det = 0
    for i in range(len(network[it][1])) : #For each neuron 

        err = 0 #Calculating err for each neuron which is summation(error[it+1][j]*w[j][i])

        for j in range(len(network[it+1][1])) :
            err += (error[it+1][j]*network[it+1][0][j][i])


        error[it][i] = err 
        delta = error[it][i]*derivative(network[it][1][i],network[it][3])  
        error[it][i] = delta
        sum_det += delta
        for j in range(len(network[it][0][i])) :
            network[it][0][i][j] += (learning_rate*delta*input[j])


    bias*= (1 + learning_rate*sum_det)
    
            
            
    
    return network
        
    

In [83]:
def normalise(input) :
    mean = sum(input) / len(input) 
    variance = sum([((x - mean) ** 2) for x in input]) / len(input)
    for k in range(len(input)) :
        input[k] = (input[k] - mean)/variance
        
    return input

In [99]:
def train(network,output,bias,learning_rate,it,epoch,num_img) :
    error = []
    
    for i in range(epoch):
        for j in range(num_img) :
            start_time = datetime.now() 
            answer = 0 
            a[j] = normalise(a[j]) 
            network = forward_prop(network,a[j],bias,it)
            network = back_prop(network,a[j],output[0],learning_rate,bias,it)
            
            #answer += error_function(network[it-1][1],output[0])
            #print("Time taken to run a is cell is  (hh:mm:ss.ms) {}".format(datetime.now() - start_time))
            b[j] = normalise(b[j]) 
            network = forward_prop(network,b[j],bias,it)
            network = back_prop(network,b[j],output[1],learning_rate,bias,it)
            #answer += error_function(network[it-1][1],output[1])
            
            #print("Time taken to run b is cell is  (hh:mm:ss.ms) {}".format(datetime.now() - start_time))
            c[j] = normalise(c[j]) 
            network = forward_prop(network,c[j],bias,it)
            network = back_prop(network,c[j],output[2],learning_rate,bias,it)
            #answer += error_function(network[it-1][1],output[2])
            #print("Time taken to run c is cell is  (hh:mm:ss.ms) {}".format(datetime.now() - start_time))
            
            d[j] = normalise(d[j] ) 
            network = forward_prop(network,d[j] ,bias,it)
            network = back_prop(network,d[j] ,output[3],learning_rate,bias,it)
            #answer += error_function(network[it-1][1],output[3])
            #print("Time taken to run d is cell is  (hh:mm:ss.ms) {}".format(datetime.now() - start_time))
            
            e[j] = normalise(e[j]) 
            network = forward_prop(network,e[j],bias,it)
            network = back_prop(network,e[j],output[4],learning_rate,bias,it)
            #answer += error_function(network[it-1][1],output[4])
            #print("Time taken to run e is cell is  (hh:mm:ss.ms) {}".format(datetime.now() - start_time))
            
            f[j] = normalise(f[j]) 
            network = forward_prop(network,f[j],bias,it)
            network = back_prop(network,f[j],output[5],learning_rate,bias,it)
            #answer += error_function(network[it-1][1],output[5])
            #print("Time taken to run e is cell is  (hh:mm:ss.ms) {}".format(datetime.now() - start_time))
            
            
            print("Image number ",j," done")
            print("Time taken to run Image ",j," is  (hh:mm:ss.ms) {}".format(datetime.now() - start_time))
            print(network[2][1])
                
        error.append(answer/epoch*num_img)
        
    
    return network,error

In [116]:
def predict(tmp,input,bias,it) :
    network = copy.deepcopy(tmp) 
    #print(it)
    my_input = input
    my_bias = bias 
    #print(my_input)
    for i in range(it-1) :  #Each hidden layer
        sum_active = [0 for i in range(len(network[i][1]))]
        for j in range(len(network[i][1])): #For each neuron 
            sum_active[j] = active_sum(network[i][0][j],my_input,my_bias)
        
        sum_active = normalise(sum_active)
        for j in range(len(network[i][1])) :
          activation = Activation(sum_active[j],network[i][3])      
          network[i][1][j] = activation
        
        my_input = network[i][1]
        my_bias = network[i][2]         
            
    #### For last layer 
    sum_active1 = [0 for i in range(len(network[it-1][1]))]
    for j in range(len(network[it-1][1])) :
        sum_active1[j] = active_sum(network[it-1][0][j],my_input,my_bias)
    
    sum_active = normalise(sum_active)
    output = Softmax(sum_active1)
    network[it-1][1] = output 
        
            
    #output = Softmax(sum_active)
    #for j in range(len(network[it-1][1])): #For each neuron 
     #   network[it-1][1][j] = output[j]
                     
    return network[it-1][1]

In [32]:
def dataset_extraction(path,class_number):
    x_train=[]
    y_train=[]
    all_images=glob.glob(path+'/*.jpg')
    for path in all_images :
            img=load_img(path, target_size=(150,150))
            img=img_to_array(img)
            img = img/255
            x_train.append(img)
            y_train.append(class_number)
    return np.array(x_train),np.array(y_train)

In [33]:
from datetime import datetime 

In [34]:
start_time = datetime.now() 
trainX_building, trainY_building  = dataset_extraction("/content/drive/My Drive/seg_test/seg_test/buildings/",0)
trainX_forest,trainY_forest  = dataset_extraction("/content/drive/My Drive/seg_test/seg_test/forest/",1)
trainX_glacier,trainY_glacier  = dataset_extraction("/content/drive/My Drive/seg_test/seg_test/glacier/",2)
trainX_mount,trainY_mount  = dataset_extraction("/content/drive/My Drive/seg_test/seg_test/mountain/",3)
trainX_sea,trainY_sea  = dataset_extraction("/content/drive/My Drive/seg_test/seg_test/sea/",4)
trainX_street,trainY_street  =  dataset_extraction("/content/drive/My Drive/seg_test/seg_test/street/",5)

print('Time taken to run this cell is  (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Time taken to run this cell is  (hh:mm:ss.ms) 0:13:15.111000


In [35]:
import matplotlib.pyplot as plt 
%matplotlib inline

import math 
from random import random 

from PIL import Image 

import copy 

In [36]:
trainX_building.shape , trainX_forest.shape , trainX_glacier.shape , trainX_mount.shape, trainX_sea.shape , trainX_street.shape   

((437, 150, 150, 3),
 (474, 150, 150, 3),
 (553, 150, 150, 3),
 (525, 150, 150, 3),
 (510, 150, 150, 3),
 (501, 150, 150, 3))

In [39]:
a = [0 for i in range(400)]
b = [0 for i in range(400)]
c = [0 for i in range(400)]
d = [0 for i in range(400)]
e = [0 for i in range(400)]
f = [0 for i in range(400)]

In [40]:
trainX_forest = trainX_forest[:400]
trainX_glacier = trainX_glacier[:400]
trainX_mount = trainX_mount[:400]
trainX_sea = trainX_sea[:400]
trainX_street = trainX_street[:400]
trainX_building = trainX_building[:400]

In [56]:
def conv_preprocess(input) :
  a = conv2d(input,filter[2]) 
  #print(len(a))
  b = conv2d(a,filter[3]) 
  #print(len(b))
  c = pooling(np.array(b)) 
  #print(c.shape)
  d = conv2d(c,filter[1])
  #print(len(d))
  e = conv2d(d,filter[0])
  #print(len(e))
  f = pooling(np.array(e)) 
  #print(f.shape)
  f = np.array(f)
  return f.flatten().reshape(f.shape[0]*f.shape[1]*f.shape[2])

In [45]:
for i in range(60) :
  start_time = datetime.now() 
  a[i] = conv_preprocess(trainX_building[i])
  b[i] = conv_preprocess(trainX_forest[i])
  c[i] = conv_preprocess(trainX_glacier[i])
  d[i] = conv_preprocess(trainX_mount[i])
  e[i] = conv_preprocess(trainX_sea[i])
  f[i] = conv_preprocess(trainX_street[i])
  print('Time taken to run',i,"th it is cell is  (hh:mm:ss.ms) {}".format(datetime.now() - start_time))
  


Time taken to run 0 th it is cell is  (hh:mm:ss.ms) 0:00:16.882990
Time taken to run 1 th it is cell is  (hh:mm:ss.ms) 0:00:16.829576
Time taken to run 2 th it is cell is  (hh:mm:ss.ms) 0:00:16.918537
Time taken to run 3 th it is cell is  (hh:mm:ss.ms) 0:00:16.876847
Time taken to run 4 th it is cell is  (hh:mm:ss.ms) 0:00:17.007204
Time taken to run 5 th it is cell is  (hh:mm:ss.ms) 0:00:16.855767
Time taken to run 6 th it is cell is  (hh:mm:ss.ms) 0:00:17.300041
Time taken to run 7 th it is cell is  (hh:mm:ss.ms) 0:00:16.876266
Time taken to run 8 th it is cell is  (hh:mm:ss.ms) 0:00:16.766655
Time taken to run 9 th it is cell is  (hh:mm:ss.ms) 0:00:16.814931
Time taken to run 10 th it is cell is  (hh:mm:ss.ms) 0:00:16.715128
Time taken to run 11 th it is cell is  (hh:mm:ss.ms) 0:00:16.810686
Time taken to run 12 th it is cell is  (hh:mm:ss.ms) 0:00:16.864050
Time taken to run 13 th it is cell is  (hh:mm:ss.ms) 0:00:16.836322
Time taken to run 14 th it is cell is  (hh:mm:ss.ms) 0:00:

In [51]:
a[0].shape 

(3072,)

In [92]:
output= [[1,0,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,0,0],[0,0,0,1,0,0],[0,0,0,0,1,0],[0,0,0,0,0,1]]

In [108]:
model = Model()

In [109]:
model.add(512,3072,'ReLU')
model.add(70,512,'ReLU')
model.add(6,70,'softmax')

In [110]:
bias = 1
epoch = 2
learning_rate = 0.1
print(model.it)

3


In [111]:
model.network[2][1]

[0.9149650443990828,
 0.6859132304306722,
 0.8718729083674617,
 0.8051485365436811,
 0.4777132993031955,
 0.8653026439525754]

In [112]:
model.network , error = train(model.network,output,bias,learning_rate,model.it,epoch,50)

Image number  0  done
Time taken to run Image  0  is  (hh:mm:ss.ms) 0:00:08.859485
[0.9999984993148466, 2.6210446041386213e-07, 5.402831901472661e-07, 1.79670114808652e-08, 6.781409627807202e-07, 2.189528393271472e-09]
Image number  1  done
Time taken to run Image  1  is  (hh:mm:ss.ms) 0:00:08.780624
[6.890741754639986e-07, 3.8398970894128154e-13, 0.9999904947743181, 1.735755669443948e-06, 7.080339103041063e-06, 5.634999110984609e-11]
Image number  2  done
Time taken to run Image  2  is  (hh:mm:ss.ms) 0:00:08.855333
[0.31669779065538683, 8.475836462441556e-05, 0.21449824658660038, 0.054440003064889324, 0.40619803514333475, 0.008081166185164397]
Image number  3  done
Time taken to run Image  3  is  (hh:mm:ss.ms) 0:00:08.774277
[0.3689529491239352, 0.46029316898095773, 0.0018751609238754712, 0.0023565181492573016, 0.006071404148429325, 0.16045079867354495]
Image number  4  done
Time taken to run Image  4  is  (hh:mm:ss.ms) 0:00:08.842133
[0.163169243084392, 0.010155844500942661, 0.015021

In [117]:
predict(model.network,a[55],bias,model.it)

[1.944567692753698e-17,
 2.1606377203714643e-25,
 1.0,
 1.743415668256082e-32,
 1.7910081500415787e-20,
 2.866135171847541e-20]

In [118]:
predict(model.network,b[55],bias,model.it)

[1.0968800766791255e-15,
 1.7851592148605872e-17,
 0.9999999999999983,
 5.468159305928717e-16,
 1.052223744072945e-18,
 6.90942821183285e-33]

In [120]:
predict(model.network,d[48],bias,model.it)

[0.011260860369035994,
 0.9868568594645517,
 0.0018637571633371645,
 1.3441138646687522e-05,
 4.939021705342326e-06,
 1.428427230429533e-07]

In [121]:
predict(model.network,f[48],bias,model.it)

[0.017035345758999534,
 1.069336828084511e-07,
 0.11946350010994375,
 1.7431456447862654e-10,
 0.8635010470230483,
 1.1024584854324685e-14]

In [124]:
predict(model.network,c[25],bias,model.it)

[0.9999996361666154,
 3.3294682850831655e-07,
 1.2416953727517308e-08,
 1.6485741266769486e-12,
 1.846260132707296e-08,
 5.352413550847635e-12]

In [98]:
np.exp(98)

3.637970947608805e+42